In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump
from pathlib import Path

DATA_DIR = Path("data")
TRAIN_PATH = DATA_DIR / "Training.csv"
TEST_PATH = DATA_DIR / "Testing.csv"

# Common column names used in the public dataset
SYMPTOM_COLS = [f"Symptom_{i}" for i in range(1, 18)]
LABEL_COL = "prognosis"

def load_and_clean(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Normalize text: lower, strip, replace spaces with underscores
    for c in SYMPTOM_COLS:
        if c in df.columns:
            df[c] = df[c].fillna("").astype(str).str.lower().str.strip().str.replace(" ", "_")
    if LABEL_COL in df.columns:
        df[LABEL_COL] = df[LABEL_COL].str.strip()
    return df

def build_symptom_vocab(df: pd.DataFrame) -> list:
    vocab = set()
    for c in SYMPTOM_COLS:
        if c in df.columns:
            vocab.update(s for s in df[c].unique() if s and s != "nan")
    vocab = sorted(vocab)
    return vocab

def rows_to_multihot(df: pd.DataFrame, vocab_idx: dict) -> np.ndarray:
    X = np.zeros((len(df), len(vocab_idx)), dtype=int)
    for i, row in df.iterrows():
        for c in SYMPTOM_COLS:
            if c in df.columns:
                s = row[c]
                if s and s != "nan" and s in vocab_idx:
                    X[i, vocab_idx[s]] = 1
    return X

def main():
    print("Loading data…")
    train_df = load_and_clean(TRAIN_PATH)
    test_df = load_and_clean(TEST_PATH)

    print("Building symptom vocabulary…")
    symptom_vocab = build_symptom_vocab(train_df)
    vocab_idx = {s: i for i, s in enumerate(symptom_vocab)}

    print("Vectorizing…")
    X_train = rows_to_multihot(train_df, vocab_idx)
    X_test = rows_to_multihot(test_df, vocab_idx)

    print("Encoding labels…")
    le = LabelEncoder()
    y_train = le.fit_transform(train_df[LABEL_COL])
    y_test = le.transform(test_df[LABEL_COL])

    print(f"Training data: {X_train.shape}, Test data: {X_test.shape}, classes: {len(le.classes_)}")

    print("Training RandomForest…")
    clf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)

    print("Evaluating…")
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    print("Saving artifacts…")
    dump(clf, "model.joblib")
    dump(le, "label_encoder.joblib")
    with open("symptom_list.json", "w") as f:
        json.dump(symptom_vocab, f, indent=2)

    # Optional: minimal precaution text per disease (extend as you like)
    disease_precautions = {
        # Add/extend safely with generic guidance (no dosages)
        "Common Cold": [
            "Rest and stay warm",
            "Drink warm fluids",
            "Consider saline gargles",
            "Seek care if symptoms persist >3 days"
        ],
    }
    # Fill all diseases with a generic fallback if not specified
    for dis in le.classes_:
        if dis not in disease_precautions:
            disease_precautions[dis] = [
                "Rest and hydrate well",
                "Monitor symptoms",
                "Seek medical advice if symptoms worsen or persist"
            ]
    with open("precautions.json", "w") as f:
        json.dump(disease_precautions, f, indent=2)

    print("Done.")

if __name__ == "__main__":
    main()
